# Recitation 2 (09/06)

## General Pytorch Operations

In [2]:
import torch
import numpy as np

# Create uinitialized tensor
x = torch.FloatTensor(2,3)

# From numpy
np_arr = np.random.random((2,3)).astype(float)
x1 = torch.FloatTensor(np_arr)
x2 = torch.randn(2,3)

# export to numpy array
x_np = x2.numpy()


# Basic operation
x = torch.arange(4,dtype = torch.float).view(2,2)
print(x)
s = torch.sum(x)
e = torch.exp(x)

#Elementwise and matrix multiplication
z = s*e + torch.matmul(x1,x2.t()) # Size 2*2

tensor([[0., 1.],
        [2., 3.]])


## Move Tensors to the GPU

### The following is a bad example
Operations between GPU and CPU tensors will fail. Operations require all arguments to be on the same device.

In [3]:
# # Create tensor
# x = torch.rand(3,2)
# # Copy to GPU
# y = x.cuda()
# # copy back to CPU
# z = y.cpu()
# try:
#     y.numpy()
# except RuntimeError as e:
#     print(e)

AssertionError: Torch not compiled with CUDA enabled

## Backpropagation

In [8]:
import torch
x = torch.tensor(torch.arange(0,4).float(),requires_grad=True)
print(x.type)
y = x**2
y.sum().backward()

print(x)
print(y)
print(x.grad)

<built-in method type of Tensor object at 0x11bf536c0>
tensor([0., 1., 2., 3.], requires_grad=True)
tensor([0., 1., 4., 9.], grad_fn=<PowBackward0>)
tensor([0., 2., 4., 6.])


/Users/robert/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


## Neural Network Modules

In [9]:
import torch
# torch.nn.Linear is a single layer perceptron
net = torch.nn.Linear(4,2)
print(net)

Linear(in_features=4, out_features=2, bias=True)


In [18]:
# the .forward() method applies the function
j = torch.arange(0,4).float()
# k = net.forward(x)
k = net(x) # Alternatively
print(k)
for param in net.parameters():
    print(param)

tensor([0.6019, 0.5914], grad_fn=<AddBackward0>)
Parameter containing:
tensor([[-0.4524,  0.3272,  0.0376, -0.0523],
        [-0.2944,  0.3686, -0.2257,  0.3013]], requires_grad=True)
Parameter containing:
tensor([ 0.3563, -0.2296], requires_grad=True)


### BAD Example of MLP

In [20]:
import torch
import torch.nn as nn
class MyNet0(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(MyNetwokWithParams,self).__init()
        self.layer1_weights = nn.Parameter(torch.randn(input_size,hidden_size))
        self.layer1_bais = nn.Parameter(torch.randn(hidden_size))
        self.layer2_weights = nn.Parameter(torch.randn(hidden_size,output_size))
        self.layer2_bias = nn.Parameter(torch.randn(outputsize))
        
    def forward(self,x):
        h1 = torch.matmul(x,self.layer1_weights)+ self.layer2_bias
        h1_act = torch.max(h1,torch.zeros(h1.size())) #ReLU
        output = torch.matmul(h1_act,self.layer2_weights) + self.layer2_bias
net = MyNet0(4,16,2)

NameError: name 'MyNetwokWithParams' is not defined

### A BETTER WAY

In [24]:
import torch
import torch.nn as nn
class MyNet1(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super().__init__()
        self.layer1 = torch.nn.Linear(input_size,hidden_size)
        self.layer2 = torch.nn.Sigmoid()
        self.layer3 = torch.nn.Linear(hidden_size,output_size)
    
    def forward(self,input_val):
        h = input_val
        h = self.layer1(h)
        h = self.layer2(h)
        h = self.layer3(h)
        return h
net = MyNet1(4,16,2)
print(net)

MyNet1(
  (layer1): Linear(in_features=4, out_features=16, bias=True)
  (layer2): Sigmoid()
  (layer3): Linear(in_features=16, out_features=2, bias=True)
)


### A EVEN BETTER WAY

In [26]:
import torch
import torch.nn as nn
def generate_net(input_size,hidden_size,output_size):
    return nn.Sequential(nn.Linear(input_size,hidden_size),nn.ReLU(),nn.Linear(hidden_size,output_size))

net = generate_net(4,16,2)
print(net)

Sequential(
  (0): Linear(in_features=4, out_features=16, bias=True)
  (1): ReLU()
  (2): Linear(in_features=16, out_features=2, bias=True)
)


### For Bigger Networks

In [27]:
def relu_mlp(size_list):
    layers = []
    for i in range(len(size_list)-2):
        layers.append(nn.Linear(size_list[i],size_list[i+1]))
        layers.append(nn.ReLU())
    layers.append(nn.Linear(size_list[-2],size_list[-1]))
    return nn.Sequential(*layers)

my_big_MLP = nn.Sequential(
    relu_mlp([100,512,512,256]),
    nn.Sigmoid(),
    relu_mlp([256,128,64,32,10]))

                            

## Optimizations and losses

### Final Layers and Losses

In [32]:
import torch
import numpy as np
# torch.nn.CrossEntropyLoss includes both softmax and loss criterion, and it is stable (uses log softmax)
x = torch.tensor([np.arange(4),np.zeros(4),np.ones(4)]).float()
y = torch.tensor([0,1,0])
criterion = nn.CrossEntropyLoss()

output = net(x)
loss = criterion(output,y)
print(loss)

tensor(0.6603, grad_fn=<NllLossBackward>)


### Use the optimiaer

subclass of **torch.nn.Optimizer**

In [33]:
# import torch
# import torch.nn.Optimizer as optimizer
# n_iter= 100
# for i in range(n_iter):
#     optimizer.zero_grad()
#     output = net(x)
#     loss = criterion(output,y)
#     loss.backward()
#     optimizer.step()

ModuleNotFoundError: No module named 'torch.nn.Optimizer'

### Saving and Loading

In [37]:
net = torch.nn.Sequential(
    torch.nn.Linear(28*28,256),
    torch.nn.Sigmoid(),
    torch.nn.Linear(256,10)
)
print(net.state_dict().keys())

# Save a dictionary
torch.save(net.state_dict(),'test.t7')
# Load a dictionary
net.load_state_dict(torch.load('test.t7'))

odict_keys(['0.weight', '0.bias', '2.weight', '2.bias'])


IncompatibleKeys(missing_keys=[], unexpected_keys=[])